# Imports

In [56]:
import pandas as pd
import numpy as np
import spreg
import libpysal
import geopandas as gpd
from libpysal.weights import Queen
from spreg import ML_Lag

# Data

In [130]:
# Read in the data
# db = libpysal.io.open("../_data/shapefile/map_municipalities.dbf","r")
db = pd.read_csv('../_data/data_municipalities_final.csv')

# Train and test datasets (test on Wielkopolskie)
db["municipality_code_str"] = db["municipality_code"].astype(str)
db.loc[:607, 'municipality_code_str'] = '0' + db.loc[:607, 'municipality_code'].astype(str)
db_train = db[db["municipality_code_str"].str[0:2] != "30"]
db_test = db[db["municipality_code_str"].str[0:2] == "30"]
# drop
db_train.drop(columns=["municipality_code_str"], inplace=True)
db_test.drop(columns=["municipality_code_str"], inplace=True)
print(db_train.shape, db_test.shape)
# Specify the path to the shapefile
shapefile_path = '../_data/shapefile/map_municipalities.shp'

# Read the shapefile
gdf = gpd.read_file(shapefile_path)

# Train and test datasets
gdf_train = gdf[gdf["mncplty_c"].str[0:2] != "30"]
gdf_test = gdf[gdf["mncplty_c"].str[0:2] == "30"]
print(gdf_train.shape, gdf_test.shape)

/tmp/ipykernel_343002/516671297.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_train.drop(columns=["municipality_code_str"], inplace=True)
/tmp/ipykernel_343002/516671297.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_test.drop(columns=["municipality_code_str"], inplace=True)


(2251, 45) (226, 45)
(2251, 3) (226, 3)


# SAR Model

## On train dataset

In [145]:
# Create a spatial weights object
w = Queen.from_dataframe(gdf_train)
ds_name = "municipalities_train"
y_name = "percent_vaccinated_log"
y = db_train[y_name].values
y.shape = (len(y),1)
x_names = db_train.columns.to_list()[4:-1]
x_names.remove("county_name")
x_names.remove("voivodeship")
x_names.remove("municipality_code_old")
x = db_train[x_names].values
w_name = "border_contiguity"
w.transform = 'r'
mllag = ML_Lag(y,x,w,name_y=y_name,name_x=x_names,name_w=w_name,name_ds=ds_name,vm=True, latex=True) 


/tmp/ipykernel_343002/1707073028.py:2: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_train)


37


In [146]:
print(mllag.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :municipalities_train
Weights matrix      :border_contiguity
Dependent Variable  :percent_vaccinated_log                Number of Observations:        2251
Mean dependent var  :      0.0650                Number of Variables   :          39
S.D. dependent var  :      0.3194                Degrees of Freedom    :        2212
Pseudo R-squared    :      0.7982
Spatial Pseudo R-squared:  0.4057
Log likelihood      :  -1048.0998
Sigma-square ML     :      0.1070                Akaike info criterion :    2174.200
S.E of regression   :      0.3272                Schwarz criterion     :    2397.246

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
----------------------------------------

### RMSE in-sample

In [168]:
y_name = "percent_vaccinated_log"
y = db_train[y_name].values
mllag.predy
rmse = np.sqrt(np.sum((y - mllag.predy)**2)/len(y))
print(rmse)

0.3271820848656099


## On test dataset

In [ ]:
# firguring out betas and rho
# first beta is intercept
# last beta is rho
mllag.betas, mllag.rho

In [172]:
# Predict on the test dataset
# Prepare y_test
y_test = db_test["percent_vaccinated_log"].values
# Create a spatial weights object
w = Queen.from_dataframe(gdf_test)
# Calcualte Wy
w_array = w.to_sparse().toarray()
row_sums = w_array.sum(axis=1)
w_array_normalized = w_array / row_sums[:, np.newaxis]
Wy = np.dot(w_array_normalized, y_test)
# Calculate y_pred
y_pred = mllag.betas[0] + np.dot(db_test[x_names].values,mllag.betas[1:-1]) + mllag.betas[-1]*Wy
# Calculate RMSE
rmse = np.sqrt(np.sum((y_test - y_pred)**2)/len(y_test))
print(rmse)

5.710633664888162


/tmp/ipykernel_343002/895585023.py:5: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_test)


# SDM

## In-sample

In [166]:
# Create a spatial weights object
w = Queen.from_dataframe(gdf_train)
ds_name = "municipalities_train"
y_name = "percent_vaccinated_log"
y = db_train[y_name].values
y.shape = (len(y),1)
x_names = db_train.columns.to_list()[4:-1]
x_names.remove("county_name")
x_names.remove("voivodeship")
x_names.remove("municipality_code_old")
x = db_train[x_names].values
w_name = "border_contiguity"
w.transform = 'r'
mllag_SDM = ML_Lag(y,x,w,slx_lags=1,name_y=y_name,name_x=x_names,name_w=w_name,name_ds=ds_name,vm=True, latex=True) 


/tmp/ipykernel_343002/1073884158.py:2: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_train)


/home/jakub/wec2024/WEC2024/.venv/lib/python3.10/site-packages/spreg/diagnostics.py:354: RuntimeWarning: invalid value encountered in sqrt
  se_result = np.sqrt(variance)
/home/jakub/wec2024/WEC2024/.venv/lib/python3.10/site-packages/spreg/diagnostics.py:170: RuntimeWarning: invalid value encountered in sqrt
  ) / np.sqrt(variance)


In [167]:
print(mllag_SDM.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG WITH SLX - SPATIAL DURBIN MODEL (METHOD = FULL)
-------------------------------------------------------------------------------------------------
Data set            :municipalities_train
Weights matrix      :border_contiguity
Dependent Variable  :percent_vaccinated_log                Number of Observations:        2251
Mean dependent var  :      0.0650                Number of Variables   :          76
S.D. dependent var  :      0.3194                Degrees of Freedom    :        2175
Pseudo R-squared    :      0.0000
Spatial Pseudo R-squared:  0.0387
Log likelihood      :  -5680.1584
Sigma-square ML     :      6.2893                Akaike info criterion :   11512.317
S.E of regression   :      2.5078                Schwarz criterion     :   11946.971

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-St